In [1]:
from os import listdir
from os.path import isfile, join
mypath = 'citibikedata/largeset'
onlyfiles = [join(mypath,f) for f in listdir(mypath) if isfile(join(mypath, f))]

In [12]:
import numpy as np
import pandas as pd
import json

# When I was doing single-file testing: FNAME = 'citibikedata/smallset/station_status.json.100'

def load_from_one_file(filepath):
    with open(filepath, "r") as read_file:
        data = json.load(read_file)
    timestamp = data['last_updated']
    # Make a dataframe for this one file, from the array of stations
    df_this_file = pd.DataFrame(data['data']['stations']).astype('int32',errors='ignore').drop(
           columns=['eightd_active_station_services','eightd_has_available_keys','num_ebikes_available'],
           errors='ignore')
    df_this_file['ts'] = timestamp
    return df_this_file

In [17]:
# In one statement, we load all the data from each individual file to construct an array of individual dataframes:
all_dataframes = [ load_from_one_file(curfile) for curfile in onlyfiles ]

In [19]:
df = pd.concat(all_dataframes, sort=False)

In [20]:
# Try to free some memory!
all_dataframes = None

In [21]:
# The shape of a dataframe is its row count x column count
df.shape

(7286096, 10)

In [22]:
df.to_pickle("citibikedata/9000timeslots.pickle")

In [19]:
# PLEASE NOTE: I was able to get the concat to work only after limiting the processing to just the first 5000 of the files.

In [33]:
# Our frame is "indexless" so we find a row via numeric index
df.iloc[0]['station_id'] == '72'

True

In [34]:
# Note that JSON is loaded in a way that makes each column be of type "object"
df.dtypes

eightd_active_station_services            object
eightd_has_available_keys                   bool
is_installed                               int64
is_renting                                 int64
is_returning                               int64
last_reported                              int64
num_bikes_available                        int64
num_bikes_disabled                         int64
num_docks_available                        int64
num_docks_disabled                         int64
num_ebikes_available                       int64
station_id                                object
ts                                         int64
most_recent_conn_DT               datetime64[ns]
dtype: object

So we have a relational DB with 2 keys:
* timestamp
* station_id

And each key combination leads to a row with several metrics, e.g.
* num_bikes_available
* num_docks_available

DataFrames are indeed 2D tables, essentially relational tables, rows and columns.

So how do we load this into a DataFrame?



In [35]:
type(df.iloc[0]['last_reported'])

numpy.int64

In [1]:
TARGETSTATION = '465'   # Station IDs are strings

In [2]:
# Houston we have a problem!
# We are getting files with "last_reported" of ZERO so those must be filtered out of the dataframe first.
df = df[df['last_reported'] > 1000]

NameError: name 'df' is not defined

In [40]:
# Converting from typical second-granularity epoch timestamp requires unit='s'
df['most_recent_conn_DT'] = pd.to_datetime(df['last_reported'], unit='s')

In [ ]:
df.dtypes

In [ ]:
df[df['station_id']==TARGETSTATION].plot(x='most_recent_conn_DT', y=['num_bikes_available'])

Let's find out exactly how much data we have.  The granularity is per minute, but what is the range?


In [172]:
df['ts'].min()

Timestamp('2018-05-26 17:23:11')

In [173]:
df['ts'].max()

Timestamp('2018-06-01 23:22:05')

In [174]:
df['ts'].max() - df['ts'].min()

Timedelta('6 days 05:58:54')